In [1]:
import os
from collections import defaultdict

In [2]:
def batchProcessing(batch, TOKEN):
    lst = batch[0].strip().split('\t')
    trainSet = int(lst[0].split()[-1])    
    cost, gamma = [float(i.split()[-1]) for i in lst[1:]]
    
    SKIP = 5 if TOKEN[-1]=='T' else 0        
    acc = float(batch[SKIP+1].split()[-1])
    posacc = float(batch[SKIP+2].split()[-1])
    total_error = int(batch[SKIP+3].strip().split()[-1])
    sum_cands = int(batch[SKIP+4].strip().split()[-1])
    max_cands = int(batch[SKIP+5].strip().split()[-1])
    min_cands = int(batch[SKIP+6].strip().split()[-1])
    mean_cands = float(batch[SKIP+7].strip().split()[-1])
    mid_cands = ''.join(batch[SKIP+8].strip().split()[1:])
    top10_cands = batch[SKIP+10].strip()
    confu_info = {'total':total_error, 'sum':sum_cands, 'max':max_cands, 'min':min_cands, 
                  'mean':mean_cands, 'mid':mid_cands, 'top10':top10_cands}
    
    pkl_file = 'cofnu_{}_{}_{}_{}_{}_{}.pkl'.format    
    ncm_pars = [0.9995, 0.99995, 0.999995, 0.9999995]
    par_str = [str(trainSet), str(cost)[2:] if cost<1 else str(cost), str(gamma)[2:]]
    pkl_files = [pkl_file(TOKEN, TOKEN, par_str[0], par_str[1], par_str[2], str(ncm_set)[2:])
                 for ncm_set in ncm_pars]
    
    batch_info = {'acc':[acc,posacc], 'confu':confu_info, 'pkl':pkl_files}
    
    return (tuple((trainSet, cost, gamma)), batch_info)

In [3]:
def sectionProcessing(file):
    with open(file, 'r', encoding='utf8') as fp:
        seqs = fp.readlines()   
        
    lst = seqs[5].split('\t')
    pre_str = 'preT' if lst[0].split()[1]=='True' else 'preF'
    nb_str = 'nbT' if lst[1].split()[1]=='True' else 'nbF'
    TOKEN = pre_str+nb_str

    START = 6
    section = dict()

    for STOP, line in enumerate(seqs[START:], START):
        if 'Output' in line:
            if STOP - START > 1:   
                key, value = batchProcessing(seqs[START:STOP], TOKEN)
                section[key] = value
            START = STOP+1  
    return (section, TOKEN)

In [4]:
def winnerSelect(section):
    train_cntpar = [1000, 2000, 3000, 4000]
    compare_mat = defaultdict(lambda :dict())
    for key, content in section.items():
        for cnt in train_cntpar:
            if cnt in key:
                compare_mat[cnt][key] = content
                break

    winner_mat = dict()
    for cnt in train_cntpar:
        winner_mat[cnt] = sorted(compare_mat[cnt].items(), key= lambda x:sum(x[1]['acc']), reverse=True)[0]
    return winner_mat

In [5]:
DATAROOT = './confusionTable/'

In [6]:
for a,b,c in os.walk(DATAROOT):        
    if not b:
        continue        
    filelist = [DATAROOT+f for f in c if 'pre' in f and 'swp' not in f]    

In [7]:
result = dict()
for file in filelist:
    print(file)
    result[file] = sectionProcessing(file)

./confusionTable/preFnbT.txt
./confusionTable/preTnbF.txt
./confusionTable/preTnbT.txt
./confusionTable/preFnbF.txt


In [8]:
resultfilename = DATAROOT+'acc_confu.csv'
with open(resultfilename, 'w', encoding='utf8') as wp:
    for section,TOKEN in result.values():
        for key, value in section.items():
            line = '{},{k[0]},{k[1]},{k[2]},{a[0]:.4f},{a[1]:.4f},{con[mean]},{con[max]}\n'.format(TOKEN, k=key, a=value['acc'], con=value['confu'])
            wp.write(line)

In [10]:
winnerfilename = DATAROOT+'acc_winner.csv'
with open(winnerfilename, 'w', encoding='utf8') as wp:
    for _,(section, TOKEN) in result.items():
        section_winner = winnerSelect(section)
        for cnt, batch in section_winner.items():
            line = '{},{k[0]},{k[1]},{k[2]},{a[0]:.4f},{a[1]:.4f},{con[mean]},{con[max]}\n'.format(
                TOKEN, k=batch[0], a=batch[1]['acc'], con=batch[1]['confu'])
            wp.write(line)